In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np
import json
from time import sleep, perf_counter as pf
import itertools as it

In [ ]:
events = pd.read_csv('datasets/specialEvents.csv')
display(events)

In [ ]:
display(events.loc[events['startDate'] != events['endDate']])

In [ ]:
output = []
for row in events.itertuples():
    for date in pd.date_range(row.startDate, row.endDate):
        output.append({
            'eventKey': row.eventKey,
            'name': row.name,
            'isOffWork': row.isOffWork,
            'type': row.type,
            'outcome': row.outcome,
            'startDate': row.startDate,
            'endDate': row.endDate,
            'startTime': row.startTime,
            'endTime': row.endTime,
            'city': row.city,
            'date': date
        })
df = pd.DataFrame(output)
df = df[['eventKey', 'name', 'isOffWork', 'type', 'outcome', 'startDate', 'endDate', 'startTime', 'endTime', 'date', 'city']]
display(df)

In [ ]:
display(df.loc[df['startDate'] != df['endDate']])

In [ ]:
df.to_csv('datasets/events_mapping.csv', index=False)

In [ ]:
df.drop(['city', 'date'], axis=1, inplace=True)
df.drop_duplicates(inplace=True)
display(df)
df.to_csv('datasets/final/events.csv', index=False)

In [ ]:
df.loc[df['eventKey'] == 161]